## Stat Functions

This notebook contains functions to conduct spectral analysis and filtering on a provided xarray dataset.

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import random
import xrft

## Spectral Function

In [ ]:
def spectral_plot(da):
    

## Filtering

In [8]:
def filter_plot(da):
    signal = da
    plt.figure(figsize=(20,15))
    plt.subplot(4,1,1)
    plt.plot(x,signal,color='black')
    plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
    plt.title('Signal')

    #Gaussian window
    win_gaus = scipy.signal.windows.gaussian(points, std=4)
    filt_gaus = scipy.signal.convolve(signal, win_gaus, mode='same')
    plt.subplot(4,1,2)
    plt.plot(x,filt_gaus)
    plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
    plt.title("Window applied: Gaussian (\u03C3=4)")

    #Exponential window
    win_exp = scipy.signal.windows.exponential(points, tau=4)
    filt_exp = scipy.signal.convolve(signal, win_exp, mode='same')
    plt.subplot(4,1,3)
    plt.plot(x,filt_exp)
    plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
    plt.title("Window applied: Exponential (\u03C4=4)")
    plt.subplots_adjust(wspace=0.4, hspace=0.4)  

    #Boxcar window
    win_av = scipy.signal.windows.boxcar(points)
    filt_av = scipy.signal.convolve(signal, win_av, mode='same')
    plt.subplot(4,1,4)
    plt.plot(x,filt_av)
    #plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
    plt.title("Window applied: Boxcar")
    plt.subplots_adjust(wspace=0.4, hspace=0.4)  

In [10]:
def gaussian_plot(da):
    plt.figure(figsize=(20,30))

    signal = y_funct
    plt.subplot(10,1,1)
    plt.plot(x,signal, color='black')
    plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
    plt.title('Signal')

    plots=[2,3,4,5]

    for n in range(2,9,2):
        window=scipy.signal.windows.gaussian(points, std=n)
        filt_gaus = scipy.signal.convolve(signal, window, mode='same')
        plt.subplot(10,1,plots[0])
        plt.plot(x,filt_gaus)
        plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
        plt.title("Gaussian window with \u03C3={}".format(n))
        plots.pop(0)

    plt.subplots_adjust(wspace=0.4, hspace=0.5) 

In [11]:
def exponential_plot(da):
    plt.figure(figsize=(20,30))

    signal = y_funct
    plt.subplot(10,1,1)
    plt.plot(x,signal, color='black')
    plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
    plt.title('Signal')

    plots=[2,3,4,5,6,7,8,9,10]

    for n in range(5,45,10):
        window=scipy.signal.windows.exponential(points, tau=n)
        filt_gaus = scipy.signal.convolve(signal, window, mode='same')
        plt.subplot(10,1,plots[0])
        plt.plot(x,filt_gaus)
        plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
        plt.title("Exponential window with \u03C4={}".format(n))
        plots.pop(0)

    plt.subplots_adjust(wspace=0.4, hspace=0.5) 

In [12]:
def boxcar_plot(da):
    signal = da
    plt.figure(figsize=(20,15))
    plt.subplot(4,1,1)
    plt.plot(x,signal,color='black')
    plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
    plt.title('Signal')
    
    win_av = scipy.signal.windows.boxcar(points)
    filt_av = scipy.signal.convolve(signal, win_av, mode='same')
    plt.subplot(2,1,2)
    plt.plot(x,filt_av)
    #plt.ylim(-int(np.max(y_funct)+10), int(np.max(y_funct)+10))
    plt.title("Window applied: Boxcar")
    plt.subplots_adjust(wspace=0.4, hspace=0.4)  